In [15]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np

import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
!pip install tflearn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127282 sha256=ec5c5e45c70cc06556aba3cceab1d2d35168d06ef836587509da0c36cafa7e76
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [18]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


Loading the Intents file

In [3]:
with open('intents (1).json') as json_data:
    intents = json.load(json_data)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Welcome to our company! How can I assist you today?',
    'Hello, thank you for reaching out. How may I help you?',
    'Hi there! How can I be of service to you?',
    'Greetings! How may I assist you?']},
  {'tag': 'company_overview',
   'patterns': ['Tell me about your company',
    'What does your company do?',
    'Can you give me an overview of your company?',
    'What services does your company provide?'],
   'responses': ['Our company is a leading provider of innovative solutions in [industry/field]. We specialize in [specific services/products]. Our goal is to deliver high-quality solutions that meet the needs of our clients and contribute to their success.',
    'At our company, we are dedicated to [industry/field] and offer a wide range of services to our clients. We strive to provide excellent solutions that drive grow

Tokenising the intents ---> get unique keywords--->perform stemming

In [19]:
words = []
classes = []
documents = []
ignore = ['?']
# to get insights from the intents file ,i tokenised the sentence in 'patterns' and appened their tagsin documents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [20]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))#set helps inremoving duplicates


classes = sorted(list(set(classes)))#same for this
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

29 documents
8 classes ['company_contact', 'company_overview', 'company_team', 'company_values', 'funny', 'goodbye', 'greeting', 'thanks']
57 unique stemmed words ['!', "'s", 'a', 'about', 'adh', 'an', 'anyon', 'ar', 'behind', 'bye', 'can', 'company', 'contact', 'day', 'detail', 'do', 'doe', 'funny', 'giv', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'inform', 'is', 'jok', 'know', 'lat', 'lot', 'mak', 'me', 'of', 'overview', 'peopl', 'principl', 'provid', 'see', 'serv', 'someth', 'team', 'tel', 'thank', 'that', 'the', 'ther', 'to', 'up', 'valu', 'what', 'who', 'with', 'yo', 'you']


bagging words for training

In [21]:

training = []
output = []
output_empty = [0] * len(classes)
for doc in documents:

    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)# ---->this creates a bag of words

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-21-a4706b1d540e>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [22]:
print(len(train_x[0]))
print(len(train_y[0]))

57
8


**NEURAL NETWORK BUILDING anad MODEL TRAINING **

In [23]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

# Building neural network with 3 layers, last one being the softmax layer
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#now train it
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.03680 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.03680 - acc: 1.0000 -- iter: 24/29
Training Step: 4000  | total loss: 0.03703 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.03703 - acc: 1.0000 -- iter: 29/29
--


Saving my trained model

In [24]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Now i'll load my saved model for using

In [25]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [26]:
# load the saved model
model.load('./model.tflearn')

Below i have written 4 functions that deals with cleaning and NLP to make it suitable to feed to the model.

In [27]:
def clean_up_sentence(sentence):# --->this helps in getting keywords in a sentence
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [29]:
ERROR_THRESHOLD = 0.1
def classify(sentence):# used to indicatethe probability of category being correct
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True) # more probable first foloowed by rest in desc order
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):#--->this fuction gets the category from 'classify' and then output using the intent file
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))# incase no match is found





In [38]:
classify('hey there!')


[('goodbye', 0.5992811), ('greeting', 0.39464253)]

In [48]:
classify("do yo foloow and values or ethics?")

[('company_values', 0.54910797),
 ('company_team', 0.2789474),
 ('company_overview', 0.16939601)]

In [52]:
response('hello ,Im abhay shanbhag')

Hi there! How can I be of service to you?


In [45]:
response ("What does the company  you do?")

We are a dynamic company focused on [industry/field]. Our team excels in delivering cutting-edge solutions and services to help our clients achieve their goals and stay ahead in the market.


In [37]:
response("What are your company values?")

At our company, we value integrity, innovation, and customer satisfaction. These principles guide our actions and decisions, ensuring that we operate ethically, drive creativity, and prioritize the needs of our clients.


In [40]:
response("How can I contact your company")

For any inquiries ,Yyu can contact our company by calling us at [phone number] or sending us an email at [email address]. We are also available on our website at [website URL], where you can find more information about our services and use our contact form to reach out to us. We look forward to hearing from you!


In [36]:
response(" tell me a joke ")


Sure, here's a joke for you: Why don't scientists trust atoms? Because they make up everything!


In [47]:
response("Nice talking to you!!")

Thank you for your interest in our company. Have a great day!
